# Optimization over SE(3)

Relies on PyTorch3D (only working on Linux)

## Install and Import Modules

Ensure `torch` and `torchvision` are installed. If `pytorch3d` is not installed, install it using the following cell:

In [2]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.11.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  404k    0  404k    0     0   733k      0 --:--:-- --:--:-- --:--:--  733k
  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-hvegdb2j
  Running command git clone -q https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-hvegdb2j
  Running command git checkout -q 2bd65027ca5c3b87b77d4f05b8eacae58d8d106f
  Resolved https://github.com/facebookresearch/pytorch3d.git to commit 2bd65027ca5c3b87b77d4f05b8eacae58d8d106f
     |████████████████████████████████| 50 kB 3.4 MB/s eta 0:00:011
     |████████████████████████████████| 661 kB 14.2 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 9.2 MB/s  eta 0:00:01
  Created wheel for pytorch3d: filename=pytorch3d-0.6.2-cp39-cp39-linux

In [8]:
# imports
import torch
from pytorch3d.transforms.so3 import (
    so3_exp_map
)

# set for reproducibility
torch.manual_seed(42)
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

## Optimization
Find the optimal transformation of vector $n_1$ to match $n_2$


In [4]:
# Define the cost function to optimize
def residual(R, t, n_1, n_2):
    return (R @ n_1 + t) - n_2

def loss(R, t, n_1, n_2):
    return torch.linalg.norm(residual(R, t, n_1, n_2))**2

In [5]:
# Define n_1 and n_2
n_1 = torch.tensor([[1], [2], [3]], dtype=torch.float32, device=device)
n_2 = torch.tensor([[-1], [2], [1]], dtype=torch.float32, device=device)

In [9]:
# Initial transformation
log_R_init = torch.randn(1, 3, dtype=torch.float32, device=device)
t_init = torch.randn(3, 1, dtype=torch.float32, device=device)

# Instantiate copy of the initialization 
log_R = log_R_init.clone().detach()
log_R.requires_grad = True
t = t_init.clone().detach()
t.requires_grad = True

init_R = so3_exp_map(log_R)
init_residual = residual(init_R, t, n_1, n_2)
init_loss = loss(init_R, t, n_1, n_2)
print(f'Init log_R: {log_R}, init t: {t}')
print(f'Initial residual: {init_residual} \n Init loss: {init_loss}')

# Init the optimizer
optimizer = torch.optim.SGD([log_R, t], lr=.1, momentum=0.9)

# run the optimization
n_iter = 100  # fix the number of iterations
for it in range(n_iter):
    # re-init the optimizer gradients
    optimizer.zero_grad()

    R = so3_exp_map(log_R)

    # compute loss
    # loss_val = loss(R, t, n_1, n_2)
    # loss_val.backward()
    r = residual(R, t, n_1, n_2)
    loss_val = torch.linalg.norm(r)**2
    loss_val.backward()
    
    # apply the gradients
    optimizer.step()

    # plot and print status message
    if it % 10==0 or it==n_iter-1:
        status = 'iteration=%3d; camera_distance=%1.3e' % (it, loss_val)
        print(f'Iteration: {it}, loss: {loss_val}')

print('Optimization finished.')


Init log_R: tensor([[ 0.1940,  2.1614, -0.1721]], device='cuda:0', requires_grad=True), init t: tensor([[ 0.1391],
        [-0.1082],
        [-0.7174]], device='cuda:0', requires_grad=True)
Initial residual: tensor([[[ 3.4046],
         [-0.6681],
         [-4.3239]]], device='cuda:0', grad_fn=<SubBackward0>) 
 Init loss: 30.733800888061523
Iteration: 0, loss: 30.733800888061523
Iteration: 10, loss: 6.842844486236572
Iteration: 20, loss: 0.43218472599983215
Iteration: 30, loss: 0.4267447292804718
Iteration: 40, loss: 0.10912315547466278
Iteration: 50, loss: 0.05448928475379944
Iteration: 60, loss: 0.01135043054819107
Iteration: 70, loss: 0.007119243964552879
Iteration: 80, loss: 0.0012975491117686033
Iteration: 90, loss: 0.0008693597628735006
Iteration: 99, loss: 0.00020798850164283067
Optimization finished.
